# Intelligent Agents: Function Execution and Alerts with LLMs

## Introducing Pushover
Setting up Pushover for notifications is quick and hassle-free! Just follow these simple steps:

### Create a Free Account
Head over to https://pushover.net and sign up.

#### Generate Your Tokens
You’ll need two tokens to enable notifications:

#### 🔑 User Token – Found on your Pushover dashboard.

#### 🛠️ Application Token – Create one by visiting https://pushover.net/apps/build and registering a new app.
(This allows you to organize notifications into separate apps later.)

#### Add to Your .env File
Store your credentials securely by adding the following to your .env:

PUSHOVER_USER=your_user_token_here

PUSHOVER_TOKEN=your_app_token_here

#### Install the Mobile App
Download the Pushover app on your phone (iOS/Android) to start receiving real-time push notifications.

In [2]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv(override=True)

True

In [3]:
# Send the Mobile Notifications
PUSH_NOTIFICATION_URI="https://api.pushover.net/1/messages.json"
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")

def push_notification(message: str):
    data={
            "token": pushover_token,
            "user": pushover_user,
            "message": message
        }
    
    response = requests.post(PUSH_NOTIFICATION_URI, data)
    if response.status_code == 200:
        return "Notification sent!"
    else:
        return f"Failed to send: {response.text}"

push_notification("Test Mobile Notification")


'Notification sent!'

These are variables are tool/function definitions in OpenAI’s function calling format. They describe to the language model what tools are available, what each tool does, and what parameters it needs.

This allows the model to:

Understand when to call a function.

Know how to call it correctly, with validated arguments.

In [4]:
# Define the Tools Functions

def record_user_details(email, name="Name not provided", notes="not provided"):
    push_notification(f"[User Interest] {name} ({email}) | Notes: {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push_notification(f"[Unknown Question] {question}")
    return {"recorded": "ok"}

# ---- Tool Schemas ----
record_user_details_json = {
    "name": "record_user_details",
    "description": "Record a user's interest using their email and optional details.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {"type": "string", "description": "User's email address"},
            "name": {"type": "string", "description": "User's name"},
            "notes": {"type": "string", "description": "Additional context or comments"},
        },
        "required": ["email"],
        "additionalProperties": False,
    },
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Log a question that the assistant couldn't answer.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {"type": "string", "description": "The unanswerable question"},
        },
        "required": ["question"],
        "additionalProperties": False,
    },
}

### Tool Dispatcher

Receives tool calls from the LLM (usually via OpenAI’s function-calling feature)

Executes the correct Python function

Returns the result in a format the model expect

You can think of it like a function router for the model's requests:

“The model says: ‘Call record_user_details with this info.’

Your code says: ‘Got it! Let me run that Python function and return the result.’”

In [5]:
# ---- Tool Dispatcher ----

def handle_tool_calls(tool_calls):
    """Execute tool calls and return their results."""
    results= []

    for call in tool_calls:
        name = call.function.name
        args = json.loads(call.function.arguments)

        print(f"Tool called: {name}")
        if name == "record_user_details":
            result = record_user_details(**args)
        elif name == "record_unknown_question":
            result = record_unknown_question(**args)
        else:
            result = {"error": f"Unknown tool: {name}"}
    
    results.append({
         "role": "tool",
         "content": json.dumps(result),
         "tool_call_id": call.id,
    })

    return results

In [6]:
globals()["record_user_details"]("jane@gmail.com.")
#print(globals())

{'recorded': 'ok'}

In [7]:
# ---- Improved Tools Dispatcher ----

# Tool function registry
TOOL_FUNCTIONS = {
    "record_user_details": record_user_details,
    "record_unknown_question": record_unknown_question,
}

def dispatch_tool_calls(tool_calls):
    """Execute tool calls and return their results."""
    results= []

    for call in tool_calls:
        name = call.function.name
        args = json.loads(call.function.arguments)
        print(f"Tool called: {name}")

        func = TOOL_FUNCTIONS.get(name)
        if func:
            try:
                result = func(**args)
            except Exception as e:
                result = {"error": f"Execution failed: {str(e)}"}
        else:
            result = {"error": f"Unknown tool: {name}"}
    
    results.append({
        "role": "tool",
        "content": json.dumps(result),
        "name": name,
        "tool_call_id": call.id,
    })

    return results

In [8]:
# Test dispatch_tool_calls function
from types import SimpleNamespace
import json

# --- Mock Tool Call Class (similar to OpenAI's tool_call format) ---
def make_tool_call(name, arguments, call_id="test-id"):
    return SimpleNamespace(
        function=SimpleNamespace(name=name, arguments=json.dumps(arguments)),
        id=call_id
    )

# --- Positive Test Case ---
tool_calls_positive =[
    make_tool_call("record_user_details", {"email": "anshulc55@icloud.com", "name": "Anshul Chauhan"}),
    make_tool_call("record_unknown_question", {"question": "What is the meaning of life?"}),
]

# --- Negative Test Case ---
tool_calls_negative = [
    make_tool_call("non_existent_tool", {"Dummy": "Test Dummy Value"}),
    make_tool_call("record_user_details", {})  # missing required "email"
]

# --- Run Tests ---
print("\n--- Positive Test Case ---")
print(dispatch_tool_calls(tool_calls_positive))

print("\n--- Negative Test Case ---")
print(dispatch_tool_calls(tool_calls_negative))


--- Positive Test Case ---
Tool called: record_user_details
Tool called: record_unknown_question
[{'role': 'tool', 'content': '{"recorded": "ok"}', 'name': 'record_unknown_question', 'tool_call_id': 'test-id'}]

--- Negative Test Case ---
Tool called: non_existent_tool
Tool called: record_user_details
[{'role': 'tool', 'content': '{"error": "Execution failed: record_user_details() missing 1 required positional argument: \'email\'"}', 'name': 'record_user_details', 'tool_call_id': 'test-id'}]


In [9]:
from pathlib import Path
from pypdf import PdfReader
project_root = Path.cwd().parent

# Read the Profile PDF
filePath = project_root / "Resourses" / "Profile.pdf"
pdfReader = PdfReader(filePath)
prof_summary = ""
for page in pdfReader.pages:
    text = page.extract_text()
    if text:
        prof_summary += text

# Read Summary file
summ_filePath = project_root / "Resourses" / "Summary.txt"
summary=""
with open(summ_filePath, "r", encoding="utf-8") as f:
    summary = f.read()

#print(prof_summary)
print(summary)

Parag Agrawal leads a balanced life that blends his demanding career with a strong emphasis on family and personal interests. He is married to Vineeta Agarwala, a general partner at the venture capital firm Andreessen Horowitz, where she invests in biotech and health tech. Together, they reside in San Francisco and are raising two children. Despite his high-profile career, Parag values quality time with his family and enjoys nurturing curiosity both personally and professionally. Outside of work, he has interests in reading, mentoring young talent, and staying connected with his cultural roots. His personal life reflects a grounded and thoughtful approach, complementing his innovative professional journey.


In [10]:
# System Prompt
name = "Parag Agrawal"
system_prompt = (
    f"You are acting as {name}, representing {name} on their website. "
    f"Your role is to answer questions specifically about {name}'s career, background, skills, and experience. "
    f"You must faithfully and accurately portray {name} in all interactions. "
    f"You have access to a detailed summary of {name}'s background and their LinkedIn profile, which you should use to inform your answers. "
    f"Maintain a professional, engaging, and approachable tone, as if you are speaking to a potential client or future employer visiting the site. "
    f"Always record any unanswered questions using the record_unknown_question tool — even if the question seems minor or unrelated."
    f"If the user shows interest or continues chatting, encourage them to share their email address. Then, use the record_user_details tool to capture their email, name (if provided), and any context worth preserving."

    f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{prof_summary}\n\n"
    f"Using this context, please converse naturally and consistently, always staying in character as {name}."
)

In [20]:
#  Handles a full LLM conversation cycle, including tool calls if needed.
from openai import OpenAI

# Load Environment Varaible
load_dotenv(override=True)

gemini_api_key=os.getenv('GEMINI_API_KEY')
gemini_base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = OpenAI(api_key=gemini_api_key, base_url=gemini_base_url)

openai_api_key=os.getenv('API_TOKEN')
deepseek_base_URL = "https://api.deepseek.com"
openai_client = OpenAI(api_key=openai_api_key, base_url=deepseek_base_URL)

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")

tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

if gemini_api_key:
    print(f"Gemini API Key exists and begins {gemini_api_key[:14]}")
else:
    print("Gemini API Key not set - please head to the troubleshooting guide in the setup folder")

def run_conversation(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    finishLoop = False # Flag to exit the loop once we get the final response
    message_obj = None # Safe initialization

    while not finishLoop:
        response = openai_client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            tools=tools,
            tool_choice="auto"  # Let the model decide
        )

        # Extract the primary reply object from the response
        message_obj = response.choices[0].message.content
        print(response.choices[0].message.tool_calls)
        

        # Check how the model finished — via tool use or a final answer
        finish_reason = response.choices[0].finish_reason
        print(f"Finish Reason : {finish_reason}")

        if finish_reason == "tool_calls":
            # Model requested tool(s) — extract the function calls
            tool_calls = response.choices[0].message.tool_calls

            # Log the tool call message in the chat history
            messages.append(message_obj)

            # Execute the tool calls and append the tool results to the messages
            tool_result = dispatch_tool_calls(tool_calls)
            messages.extend(tool_result)
            finishLoop = True
        else:
            # No more tool calls — break the loop and return final response
            finishLoop = True

    # Return the model's final response after all tool handling
    return message_obj

OpenAI API Key exists and begins sk-ba892fdbe19
Gemini API Key exists and begins AIzaSyAd8cWuxj


In [21]:
import gradio as gr

gr.ChatInterface(run_conversation, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


None
Finish Reason : stop
None
Finish Reason : stop
None
Finish Reason : stop
[ChatCompletionMessageToolCall(id='call_0_c2fa30ce-ee00-480d-ace9-3cd85855b8b4', function=Function(arguments='{"email":"peter@yahoo.com","notes":"Interested in football and hobbies."}', name='record_user_details'), type='function', index=0)]
Finish Reason : tool_calls
Tool called: record_user_details
None
Finish Reason : stop
[ChatCompletionMessageToolCall(id='call_0_248e592e-78d9-4ed9-aeef-e84ec629825c', function=Function(arguments='{"question":"What is the temperature of the Sun?"}', name='record_unknown_question'), type='function', index=0)]
Finish Reason : tool_calls
Tool called: record_unknown_question
None
Finish Reason : stop


In [1]:
from agents import Agent

agent = Agent(
    system_message="You are a helpful research assistant.",
    tools=[],
)

response = await agent.run("Summarize in short, the impact of AI in education.")
print(response)

TypeError: Agent.__init__() got an unexpected keyword argument 'system_message'